<a href="https://colab.research.google.com/github/jbmmbj/m494/blob/main/progress%2010-3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install FrEIA

In [3]:
pip install numpy

In [4]:
pip install pandas

In [5]:
# standard imports
import torch
import torch.nn as nn
from sklearn.datasets import make_moons

# FrEIA imports
import FrEIA.framework as Ff
import FrEIA.modules as Fm

BATCHSIZE = 100
N_DIM = 2

# we define a subnet for use inside an affine coupling block
# for more detailed information see the full tutorial
def subnet_fc(dims_in, dims_out):
    return nn.Sequential(nn.Linear(dims_in, 512), nn.ReLU(),
                         nn.Linear(512,  dims_out))

# a simple chain of operations is collected by ReversibleSequential
inn = Ff.SequenceINN(N_DIM)
for k in range(8):
    inn.append(Fm.AllInOneBlock, subnet_constructor=subnet_fc, permute_soft=True)

optimizer = torch.optim.Adam(inn.parameters(), lr=0.001)

# a very basic training loop
for i in range(1000):
    optimizer.zero_grad()
    # sample data from the moons distribution
    data, label = make_moons(n_samples=BATCHSIZE, noise=0.05)
    x = torch.Tensor(data)
    # pass to INN and get transformed variable z and log Jacobian determinant
    z, log_jac_det = inn(x)
    # calculate the negative log-likelihood of the model with a standard normal prior
    loss = 0.5*torch.sum(z**2, 1) - log_jac_det
    loss = loss.mean() / N_DIM
    # backpropagate and update the weights
    loss.backward()
    optimizer.step()

# sample from the INN by sampling from a standard normal and transforming
# it in the reverse direction
z = torch.randn(BATCHSIZE, N_DIM)
samples, _ = inn(z, rev=True)

In [6]:
from sklearn.datasets import fetch_openml
# Load data from https://www.openml.org/d/554
x, y = fetch_openml('mnist_784', version=1, return_X_y=True)

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [7]:
x.shape
type(x)
type(data)
x = x.to_numpy()
x = torch.Tensor(x)
x.shape

torch.Size([70000, 784])

In [1]:
import torch.nn as nn
import FrEIA.framework as Ff
import FrEIA.modules as Fm

def subnet_fc(dims_in, dims_out):
    '''Return a feed-forward subnetwork, to be used in the coupling blocks below'''
    return nn.Sequential(nn.Linear(dims_in, 128), nn.ReLU(),
                         nn.Linear(128,  128), nn.ReLU(),
                         nn.Linear(128,  dims_out))

# a tuple of the input data dimension. 784 is the dimension of flattened MNIST images.
# (input_dims would be (3, 32, 32) for CIFAR for instance)
input_dims = (784,)

# construct the INN (not containing any operations so far)
inn = Ff.SequenceINN(*input_dims)

# append coupling blocks to the sequence of operations
for k in range(8):
    inn.append(Fm.AllInOneBlock, subnet_constructor=subnet_fc)

z, jac = inn(x)
x_inv, jac_inv = inn(z, rev=True)

# inverting from the outputs should give the original inputs again
assert torch.max(torch.abs(x - x_inv)) < 1e-5

# the inverse log-det-Jacobian should be the negative of the forward log-det-Jacobian
assert torch.max(torch.abs(jac + jac_inv)) < 1e-5

cond_dims = (10,)

# use the input_dims (784,) from above
cinn = Ff.SequenceINN(*input_dims)

for k in range(8):
    # The cond=0 argument tells the operation which of the conditions it should
    # use, that are supplied with the call. So cond=0 means 'use the first condition'
    # (there is only one condition in this case).
    cinn.append(Fm.AllInOneBlock, cond=0, cond_shape=cond_dims, subnet_constructor=subnet_fc)

# the conditions have to be given as a list (in this example, a list with
# one entry, 'one_hot_labels').  In general, multiple conditions can be
# given. The cond argument of the append() method above specifies which
# condition is used for each operation.
z, jac = cinn(x, c=[one_hot_labels])


NameError: ignored